<a href="https://colab.research.google.com/github/JMML2021/0401_Registre-de-logs/blob/main/0401_Registre-de-logs%20Rev%20B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# importing libraries
from urllib.request import urlretrieve
import pandas as pd
import os
import re
from datetime import datetime
import pytz
#import geocoder
import folium
import time
import urllib.request
import json
import geopandas
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.0 MB/s 
     |████████████████████████████████| 6.3 MB 23.3 MB/s 
     |████████████████████████████████| 16.7 MB 234 kB/s 


In [4]:
# Activo Google Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).




```
# Tiene formato de código
```

# Nivell 1
L'analista ha d'assegurar-se que els registres consisteixen en una gamma completa de missatges i s'interpreten segons el context. Els elements de registre han d'estandaritzar-se, utilitzant els mateixos termes o terminologia, per evitar confusions i proporcionar cohesió.

Com Científic de Dades se t'ha proporcionat accés als registres-Logs on queda registrada l'activitat de totes les visites a realitzades a la pàgina web de l'agència de viatges "akumenius.com".


In [5]:
# Abro archivo

df = list(open('/content/drive/MyDrive/Web_access_log-akumenius.com.txt'))
df[200:210]

['www.akumenius.com 180.76.5.31 - - [23/Feb/2014:03:15:03 +0100] "GET /destinos-baratos/destinos-caracteristicas/hoteles-baratos-en-Zurich_SUIZA-con-Alquiler-de-silla-de-ruedas HTTP/1.1" 200 8360 "-" "Mozilla/5.0 (compatible; Baiduspider/2.0; +http://www.baidu.com/search/spider.html)" VLOG=-\n',
 'www.akumenius.com 66.249.76.216 - - [23/Feb/2014:03:15:03 +0100] "GET /hoteles-baratos/ofertas-hotel-Capitolio-Apartamentos-Turisticos-en-Merida-207271b-destinos.html HTTP/1.1" 404 3100 "-" "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)" VLOG=-\n',
 'www.akumenius.com 66.249.76.216 - - [23/Feb/2014:03:15:06 +0100] "GET /destinos-baratos/destinosEstrelles/hoteles-en-Gary_ESTADOS%20UNIDOS-con-4-estrellas HTTP/1.1" 200 8812 "-" "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)" VLOG=-\n',
 'www.akumenius.com 66.249.76.216 - - [23/Feb/2014:03:15:06 +0100] "GET /hoteles-baratos/ofertas-hotel-Guitart-Termes-La-Collada-en-La%20Molina-6833b-destinos.

In [ ]:
print('NUmero de registros:')
df.shape()

# Exercici 1
Estandaritza, identifica i enumera cada un dels atributs / variables de l'estructura de l'arxiu "Web_access_log-akumenius.com" que trobaràs al repositori de GitHub "Data-sources".

In [6]:
#2.- Separo la parte central.

def custom_split(str_to_split):
    separatorsCentro = "[", "]", "\s", ",",' ','+', ')', '(', '"'
    # create regular expression dynamically
    regular_exp = '|'.join(map(re.escape, separatorsCentro))

    return re.split(regular_exp, str_to_split)

In [7]:
def arregloFecha(x):
    '''
    Parses datetime with timezone formatted as:
        `[day/month/year:hour:minute:second zone]`

    Example:
        `>>> parse_datetime('13/Nov/2015:11:45:42 +0000')`
        `datetime.datetime(2015, 11, 3, 11, 45, 4, tzinfo=<UTC>)`

    Due to problems parsing the timezone (`%z`) with `datetime.strptime`, the
    timezone will be obtained using the `pytz` library.
    '''
    dt = datetime.strptime(x, '%d/%b/%Y:%H:%M:%S')

    #dt = datetime.strptime(x[0:-9], '%d/%b/%Y:%H:%M:%S')
    #print('Fecha ', dt)
    #dt_tz = int(x[-6:-3])*60+int(x[-3:-1])
    #return dt.replace(tzinfo=pytz.FixedOffset(dt_tz))
    return dt

In [22]:
import re
ladoIzq=[]
ladoDcha=[]
texto=[]

  
df1 = df.copy()
data = {'virtual_host': [],    'IPs': [],    'Fecha': [],    'request': [],  'status': [], 'size': [], 'referer': [], 'user_agent': []}

df_salida = pd.DataFrame(data)
print('lineas total ------> ', len(df1))

#for s, textoLinea  in enumerate(df1):
#  print('Hola', s, textoLinea)
for s, textoLinea  in enumerate(df1):
  #if len(textoLinea) == 23:
    #print ('---', s, textoLinea) 
  problematico= 0


  if s> problematico and s <= len(df1) :
    
    texto=custom_split(textoLinea)
    #print(s, len(texto), '<---', textoLinea) 
    #print( texto[5])
    #print('longitud', len(texto))    
    if len(texto)> 25:
      #print('hola')
      nueva_fila = {'virtual_host': texto[0],
          'IPs': texto[1],
          'Fecha': arregloFecha(texto[5]),  # Lo tranformo en fecha
          'request': texto[11],
          'status': texto[14],
          'size': texto[15], 
          'referer': texto[20]}

    else:
      print (s,'MALO    long=',  len(texto), textoLinea)
      for p,q in enumerate(texto):
        print(p,q)

    if len(texto)> 23:            
          nueva_fila = {'user_agent': texto[23]}
    else:
      nueva_fila = {'user_agent': []}

    if len(texto)> 25:      
      nueva_fila = {'Navegador': texto[25]}
    else:
      nueva_fila = {'Navegador': []}


    df_salida = df_salida.append(nueva_fila, ignore_index=True) 
  #if len(texto) <=23:
    
    
        
# Guardo la informacion en drive   
path = ('/content/drive/MyDrive/01_COLAB/' +'direcionesIP.csv')
df_salida.to_csv(path)
#
#df_salida.type
df_salida.shape


Se han truncado las últimas 5000 líneas del flujo de salida.
4 
5 24/Feb/2014:09:21:01
6 
7 0100
8 
9 
10 GET
11 /favicon.ico
12 HTTP/1.1
13 
14 200
15 3175
16 
17 -
18 
19 
20 Mercury/810
21 CFNetwork/672.0.8
22 Darwin/14.0.0
23 
24 VLOG=-

50275 MALO    long= 23 akumenius.com 69.61.47.114 - - [24/Feb/2014:09:24:32 +0100] "HEAD / HTTP/1.1" 301 - "-" "-" VLOG=-

0 akumenius.com
1 69.61.47.114
2 -
3 -
4 
5 24/Feb/2014:09:24:32
6 
7 0100
8 
9 
10 HEAD
11 /
12 HTTP/1.1
13 
14 301
15 -
16 
17 -
18 
19 
20 -
21 
22 VLOG=-

50354 MALO    long= 23 www.akumenius.com 192.185.83.114 - - [24/Feb/2014:09:25:39 +0100] "GET / HTTP/1.0" 200 40691 "-" "-" VLOG=-

0 www.akumenius.com
1 192.185.83.114
2 -
3 -
4 
5 24/Feb/2014:09:25:39
6 
7 0100
8 
9 
10 GET
11 /
12 HTTP/1.0
13 
14 200
15 40691
16 
17 -
18 
19 
20 -
21 
22 VLOG=-

52518 MALO    long= 21 localhost 180.76.5.214 - - [24/Feb/2014:10:23:25 +0100] "-" 408 - "-" "-" VLOG=-

0 localhost
1 180.76.5.214
2 -
3 -
4 
5 24/Feb/2014:10:23:25
6 
7 0100


KeyboardInterrupt: ignored

In [ ]:
type(df_salida)
df_salida.dtypes

virtual_host    float64
IPs             float64
Fecha           float64
request         float64
status          float64
size            float64
referer         float64
user_agent      float64
dtype: object

In [ ]:
df_salida.shape


(0, 8)

In [ ]:
df=  pd.read_csv(path)

In [ ]:
df_salida['IPs']

Series([], Name: IPs, dtype: float64)

# Exercici 2
Neteja, preprocesa, estructura i transforma (dataframe) les dades del registre d'Accés a la web.

In [ ]:
data_ip =df["IPs"].value_counts().rename_axis('ip').reset_index(name="visits")
data_ip.head()

,ip,visits


In [ ]:
ip = geocoder.ip("80.28.221.123")
print(ip.city)
print(ip)
print(ip.latlng)

NameError: ignored

In [ ]:
len(df_salida)

99

In [ ]:
df_salida

,Direccion,IPs,Fecha,Navegador
0,/hoteles-baratos/ofertas-hotel-Capitolio-Apart...,66.249.76.216,23/Feb/2014:03:15:03,NaN
1,/destinos-baratos/destinosEstrelles/hoteles-en...,66.249.76.216,23/Feb/2014:03:15:06,NaN
2,/hoteles-baratos/ofertas-hotel-Guitart-Termes-...,66.249.76.216,23/Feb/2014:03:15:06,NaN
3,/hoteles-baratos/ofertas-hotel-Sol-Palmeras-en...,66.249.76.216,23/Feb/2014:03:15:08,NaN
4,/hoteles-baratos/ofertas-hotel-The-Suites-At-S...,66.249.76.216,23/Feb/2014:03:15:10,NaN
...,...,...,...,...
94,/destinos-baratos/destinosEstrelles/hoteles-en...,66.249.76.216,23/Feb/2014:03:17:26,NaN
95,/hoteles-baratos/ofertas-hotel-Suite-Monte-Gol...,66.249.76.216,23/Feb/2014:03:17:27,NaN
96,/hoteles-baratos/ofertas-hotel-Barcelo-Family-...,66.249.76.216,23/Feb/2014:03:17:29,NaN
97,/hoteles-baratos/ofertas-hotel-Sofitel-Santa-C...,66.249.76.216,23/Feb/2014:03:17:32,NaN


# Exercici 3
Geolocalitza les IP's.

In [ ]:

location = geocoder.ip('66.249.76.216')

NameError: ignored

In [ ]:
#! pip install folium

In [ ]:
#! pip install geocoder

In [ ]:
import geocoder
for s, t in enumerate(df_salida['IPs']):
  #print(s,t)
  location = geocoder.ip(t)
  #print(s,t, location)
  ip = geocoder.ip(t)
  print(s, ip, ip.latlng)
  #location = ip.latlng
  #map = folium.Map(location=location, zoom_start=105)
  #folium.CircleMarker(location=location, radius=15, color="blue").add_to(map)
  #folium.Marker(location).add_to(map)
  #log, lat = location
  #print(log, lat)
  #print('-->',location)
'''
  folium.Map(location=[45.5236, -122.6750],
           tiles='Mapbox',
           API_key='your.API.key')
map
  map  

  '''#
'''
  location = ip.t

  
  
map
'''

Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json


0 <[ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json] Ipinfo - Geocode [empty]> None


Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://

1 <[ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json] Ipinfo - Geocode [empty]> None
2 <[ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json] Ipinfo - Geocode [empty]> None
3 <[ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json] Ipinfo - Geocode [empty]> None
4 <[ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json] Ipinfo - Geocode [empty]> None
5 <[ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json] Ipinfo - Geocode [empty]> None


Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://

...

Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://

88 <[ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json] Ipinfo - Geocode [empty]> None
89 <[ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json] Ipinfo - Geocode [empty]> None
90 <[ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json] Ipinfo - Geocode [empty]> None
91 <[ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json] Ipinfo - Geocode [empty]> None
92 <[ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json] Ipinfo - Geocode [empty]> None
93 <[ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json] Ipinfo - Geocode [empty]> None
94 <[ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json] Ipinfo - Geocode [empty]> None


Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json
Status code 429 from http://ipinfo.io/66.249.76.216/json: ERROR - 429 Client Error: Too Many Requests for url: http://

95 <[ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json] Ipinfo - Geocode [empty]> None
96 <[ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json] Ipinfo - Geocode [empty]> None
97 <[ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json] Ipinfo - Geocode [empty]> None
98 <[ERROR - 429 Client Error: Too Many Requests for url: http://ipinfo.io/66.249.76.216/json] Ipinfo - Geocode [empty]> None


'\n  location = ip.t\n\n  \n  \nmap\n'

In [ ]:
folium.Map(location=[45.5236, -122.6750],
           tiles='Mapbox',
           API_key='your.API.key')
map

In [ ]:
https://datasciencesphere.com/project/track-location-ip-address-python-geocoder/

# Exercici 3
Mostra'm la teva creativitat, Sorprèn-me fes un pas més enllà amb l'anàlisi anterior.

https://datasciencesphere.com/project/track-location-ip-address-python-geocoder/

In [ ]:
ip = geocoder.ip("me")
ip

print(ip.latlng)

<[OK] Ipinfo - Geocode [North Charleston, South Carolina, US]>

In [ ]:
g = geocoder.ip('74.125.226.99')
>>> g.address

In [ ]:
import requests

url = 'https://maps.googleapis.com/maps/api/geocode/json'
r = requests.get(url, params={
    'sensor': 'false', 
    'address': 'Centre Pompidou, Paris, FR'
})

results = r.json()['results']
results
#location = results[0]['geometry']['location']
#lat, lng = location['lat'], location['lng']

[]

https://janakiev.com/blog/geocoding-in-python/

In [ ]:
location = geocoder.ip("66.249.76.216")
print(ip.city)
print(ip)
print(ip.latlng)

location = ip.latlng
lat, lng = location[0], location[1]

#location = results[0]['geometry']['location']
#lat, lng = location['lat'], location['lng']

print(lat, lng, location)

NameError: ignored